## Goal: French to English Translation with RNN

## Download dataset from Kaggle

#### https://www.kaggle.com/dhruvildave/en-fr-translation-dataset

In [ ]:
! mkdir ~/.kaggle

In [ ]:
# Upload your kaggle.json file, place into the content folder
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets download dhruvildave/en-fr-translation-dataset --force

In [ ]:
# Should have zipped file here
! ls

In [ ]:
! unzip en-fr-translation-dataset.zip

## Imports required

In [ ]:
! pip install -q tensorflow_text

In [ ]:
from sklearn.utils import shuffle
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import tensorflow_text as tf_text

sns.set_style("whitegrid")

## Read dataset functions

In [ ]:
def get_total_chunks(df_chunk):
  """ Returns count of chunk in dataset. """
  total_chunks = 0
  for _, _ in enumerate(df_chunk):
    total_chunks += 1
  return total_chunks

def get_dataset_chunk(chunk_size, num_chunks, filename, all_chunks=False):
  """ Returns pandas DataFrame with a size of num_chunks. """
  df_chunk = pd.read_csv(filename, chunksize=chunk_size)
  if (all_chunks):
    total_chunks = get_total_chunks(df_chunk)
    return pd.DataFrame(df_chunk.get_chunk(total_chunks))
  else:
    return pd.DataFrame(df_chunk.get_chunk(num_chunks))

## Preprocess data functions

In [ ]:
def get_tensor_data(df):
  """ Returns tensors for input and target data. """
  tensor_data_input = tf.convert_to_tensor(np.array(df['fr'].astype(str)))
  tensor_data_target = tf.convert_to_tensor(np.array(df['en'].astype(str)))
  return tensor_data_input, tensor_data_target

# Source: https://www.tensorflow.org/text/tutorials/nmt_with_attention
def tf_lower_and_split_punct(text):
  """ Used in text processors.
      Split accented characters. 
      Keep space, a to z, and select punctuation, add spaces around punctuation, strip whitespace.
  """
  text = tf_text.normalize_utf8(text, 'NFKD')
  text = tf.strings.lower(text)
  text = tf.strings.regex_replace(text, '[^ a-z.?!,¿]', '')
  text = tf.strings.regex_replace(text, '[.?!,¿]', r' \0 ')
  text = tf.strings.strip(text)
  text = tf.strings.join(['[START]', text, '[END]'], separator=' ')
  return text

def get_text_processor():
  """ Returns TensorFlow text processors for input and target. """
  max_vocab_size = 50000

  input_text_processor = tf.keras.layers.experimental.preprocessing.TextVectorization(
      standardize=tf_lower_and_split_punct,
      max_tokens=max_vocab_size
  )

  target_text_processor = tf.keras.layers.experimental.preprocessing.TextVectorization(
      standardize=tf_lower_and_split_punct,
      max_tokens=max_vocab_size
  )

  return input_text_processor, target_text_processor

def init_text_processors(df):
  """ Initializes text processor and returns texts processors and tensors for input and target data. """
  tensor_data_input, tensor_data_target = get_tensor_data(df)
  input_text_processor, target_text_processor = get_text_processor()

  input_text_processor.adapt(tensor_data_input)
  target_text_processor.adapt(tensor_data_target)
  
  return input_text_processor, target_text_processor, tensor_data_input, tensor_data_target

def get_dataset(filename, chunksize, num_chunks):
  """ Calls functions that returns pandas DataFrame of original dataset. """
  return get_dataset_chunk(chunk_size=chunksize, num_chunks=num_chunks, filename=filename, all_chunks=False)

def filter_dataset_by_string_length(df, min_length, max_length):
  """ Returns filtered dataset based on string length. """
  df = df.loc[(df['en'].str.len() <= max_length) & (df['en'].str.len() >= min_length) & (df['fr'].str.len() <= max_length) & (df['fr'].str.len() >= min_length)]
  df = shuffle(df)
  df.reset_index(drop=True, inplace=True)
  return df

def generate_test_train(input, target, test_size=0.2):
  """ Splits data for training and testing. """
  split_at = math.ceil(input.shape[0] * test_size)
  input_test = input[:split_at]
  target_test = target[:split_at]
  input_train = input[split_at:]
  target_train = target[split_at:]
  return input_train, input_test, target_train, target_test

def preprocess_data(df, row_length):
  """ Returns one-hot encoded input and target tensors, depth, and output_size. """
  input_text_processor, target_text_processor, tensor_data_input, tensor_data_target = init_text_processors(df.iloc[:row_length])

  vocab_size = max(input_text_processor.vocabulary_size(), target_text_processor.vocabulary_size())
  input_vector = tf.one_hot(input_text_processor(tensor_data_input).numpy(), depth=vocab_size)
  target_vector = tf.one_hot(target_text_processor(tensor_data_target).numpy(), depth=vocab_size)
  output_size = target_vector.numpy().shape[1]

  return input_vector, target_vector, vocab_size, output_size 

## RNN Functions

In [ ]:
def gradient_clip(dWaa, dWax, dWya, dBa, dBy, min_value, max_value):
  """ Returns matrices clipped to values between min_value and max_value. """
  [np.clip(i, min_value, max_value, out=i) for i in [dWaa, dWax, dWya, dBa, dBy]]
  return dWaa, dWax, dWya, dBa, dBy

def init_forward_params(vocab_size, bias, output_size):
  """ Returns initialized values for forward pass. """
  Waa = np.random.rand(vocab_size, vocab_size) - bias
  Wax = np.random.rand(vocab_size, vocab_size) - bias
  Wya = np.random.rand(output_size, vocab_size) - bias
  Ba = np.zeros((vocab_size, 1))
  By = np.zeros((output_size, 1))
  return Waa, Wax, Wya, Ba, By

def init_backward_params(depth, output_size, Yt, At_p, Wya):
  """ Returns initialied values for backward pass. """
  dWaa = np.zeros((depth, depth))
  dWax = np.zeros((depth, depth))
  dBa = np.zeros((depth, 1))
  dWya = np.matmul(Yt, At_p[len(At_p)-1].T)
  dAt = np.matmul(Wya.T, Yt)
  return dWaa, dWax, dBa, dWya, Yt, dAt

def sigmoid(x):
  """ Sigmoid activation function. """
  return 1 / (1 + np.exp(-x))

def tanh(x):
  """ Tanh activation function. """
  return np.tanh(x)

def relu(x):
  """ ReLU activation function.  
      NumPy's maximum function returns element-wise maxima.
  """
  return np.maximum(0, x)

def activation(x, activation_func):
  """ Returns proper activation function given activation_func. """
  if (activation_func == 'tanh'):
    return tanh(x)
  elif (activation_func == 'relu'):
    return relu(x)
  elif (activation_func == 'sigmoid'):
    return sigmoid(x)

def dActivation(dAt, At_p, activation_func):
  """ Returns derivative of activation function given activation_func. """
  if (activation_func == 'tanh'):
    return np.multiply((1 - np.power(At_p, 2)), dAt)
  elif (activation_func == 'relu'):
    At_p[At_p < 0] = 0.0
    At_p[At_p > 0] = 1.0
    return At_p
  elif (activation_func == 'sigmoid'):
    return sigmoid(At_p) * (1 - sigmoid(At_p))

def forward(input, Waa, Wax, Wya, Ba, By, At, activation_func):
  """ Forward pass of batched input.
      Applies activation function given activation_func.
      Applies Softmax to output Yt.
      Stores each At to use in backward pass.

      Based on: At = f(Waa . At-1 + Wax . Xt + ba)
                Yt = g(Wya . At + by)
  """
  At_p = []
  At_p.append(At)

  for Xt in input:
    At = np.matmul(Waa, At) + np.matmul(Wax, Xt) + Ba
    At = activation(At, activation_func)
    At_p.append(At)

  Yt = tf.nn.softmax(np.dot(Wya, At) + By).numpy()
  return Yt, At_p

def backward(Yt, vocab_size, output_size, At_p, Xt, Waa, Wya, activation_func):
  """ Backward pass of batched input through time.
      Uses derivative of activation function given activation_func.
      Initializes values before reversed iteration.

      Based on: dL / dW, at some time step.
                dWaa += dF(At) . f(At-1)
                dWax += dF(At) . Xt-1
                dBa += dF(At)
                dBy = softmax(Yt)
  """
  dWaa, dWax, dBa, dWya, dBy, dAt = init_backward_params(vocab_size, output_size, Yt, At_p, Wya)

  for time in reversed(range(1, len(At_p))):
    dWaa = dWaa + np.matmul(dActivation(dAt, At_p[time], activation_func), At_p[time - 1].T)
    dWax = dWax + np.matmul(dActivation(dAt, At_p[time], activation_func), Xt[time - 1].T)
    dBa = dBa + dActivation(dAt, At_p[time], activation_func)
    dAt = np.matmul(Waa.T, dActivation(dAt, At_p[time], activation_func))

  return dWaa, dWax, dWya, dBa, dBy

def update_weights(dWaa, dWax, dWya, dBa, dBy, Waa, Wax, Wya, Ba, By, learning_rate):
  """ Returns updated weight values. """
  Waa = Waa - np.multiply(learning_rate, dWaa)
  Wax = Wax - np.multiply(learning_rate, dWax)
  Wya = Wya - np.multiply(learning_rate, dWya)
  Ba = Ba - np.multiply(learning_rate, dBa)
  By = By - np.multiply(learning_rate, dBy)

  return Waa, Wax, Wya, Ba, By

def rmse(target, Yt):
  """ Returns RMSE. """
  return np.sqrt(np.mean(np.power((Yt - target), 2)))

## Plotting Function

In [ ]:
def lineplot(value_list, title, sub_title, plot_type, y_label, x_label):
  """ Shows lineplot for given values in value_list. """
  x = np.arange(start=1, stop=len(value_list)+1)

  if (plot_type == 'Accuracy'):
    y = [i * 100 for i in value_list] 
  elif (plot_type == 'Loss' or plot_type == 'RMSE'):
    y = value_list

  fig, ax = plt.subplots(figsize=(12, 12))
  ax.plot(x, y, lw=3)
  ax.set_xlabel(x_label, fontsize=14)
  ax.set_ylabel(y_label, fontsize=14)
  ax.set_xlim(1, len(value_list))
  plt.suptitle(title, y=0.92, fontsize=18)
  plt.title(sub_title, fontsize=12)
  plt.show()

## Create RNN Model

In [ ]:
def generate_model(df, activation_func, learning_rate, epochs, init_matrix_bias, row_length, test_size, min_gradient_clip, max_gradient_clip):
  """ Creates Simple RNN model. 
      Calls functions to get training and testing data.
      Does forward and backward pass on training data.
      Does forward pass on testing data.
      Uses TensorFlow to get loss and accuracy values.
      Returns lists of accuracy, RMSE, and loss values for testing and training data.
  """
  df = shuffle(df)
  df.reset_index(drop=True, inplace=True)
  input_vector, target_vector, vocab_size, output_size = preprocess_data(df, row_length)
  input_train, input_test, target_train, target_test = generate_test_train(input_vector.numpy(), target_vector.numpy(), test_size)

  Waa, Wax, Wya, Ba, By = init_forward_params(vocab_size, init_matrix_bias, output_size)

  loss = tf.keras.losses.CategoricalCrossentropy()
  accuracy = tf.keras.metrics.CategoricalAccuracy()

  train_loss_list = []
  train_acc_list = []
  train_rmse_list = []

  test_loss_list = []
  test_acc_list = []
  test_rmse_list = []

  for _ in range(epochs):
    train_loss = 0
    test_loss = 0
    train_rmse = 0
    test_rmse = 0

    for train_input, train_target in zip(input_train, target_train):
      At = np.zeros((vocab_size, vocab_size))
      Yt, At_p = forward(train_input, Waa, Wax, Wya, Ba, By, At, activation_func)
      train_loss += loss(train_target, Yt)
      train_rmse += rmse(train_target, Yt)
      accuracy.update_state(train_target[:, 0], Yt[:, 0])
      dWaa, dWax, dWya, dBa, dBy = backward(Yt, vocab_size, output_size, At_p, train_input, Waa, Wya, activation_func)
      dWaa, dWax, dWya, dBa, dBy = gradient_clip(dWaa, dWax, dWya, dBa, dBy, min_gradient_clip, max_gradient_clip)
      Waa, Wax, Wya, Ba, By = update_weights(dWaa, dWax, dWya, dBa, dBy, Waa, Wax, Wya, Ba, By, learning_rate)

    train_acc_list.append(accuracy.result().numpy())
    train_loss_list.append(train_loss.numpy())
    train_rmse_list.append(train_rmse)
    
    for test_input, test_target in zip(input_test, target_test):
      At = np.zeros((vocab_size, vocab_size))
      Yt, _ = forward(test_input, Waa, Wax, Wya, Ba, By, At, activation_func)
      test_loss += loss(test_target, Yt)
      test_rmse += rmse(test_target, Yt)
      accuracy.update_state(test_target[:, 0], Yt[:, 0])

    test_acc_list.append(accuracy.result().numpy())
    test_loss_list.append(test_loss.numpy())
    test_rmse_list.append(test_rmse)

  return train_acc_list, test_acc_list, train_loss_list, test_loss_list, train_rmse_list, test_rmse_list, vocab_size, output_size

## Main Driver

In [ ]:
def init_df(filename, chunksize, num_chunks):
  """ Returns original dataset. """
  return get_dataset(filename, chunksize, num_chunks)

In [ ]:
# Get original data (number of rows = number of chunks), but row length filtering happens in preprocess_data()
filename = 'en-fr.csv'
chunksize = 500000
num_chunks = 1500000
df_ = init_df(filename, chunksize, num_chunks)

In [ ]:
# Set min and max string length to filter dataset on
min_length = 0
max_length = 20
df = filter_dataset_by_string_length(df_, min_length, max_length)
df.shape

In [ ]:
# Model Details
activation_used = 'sigmoid' # relu, tanh, or sigmoid
learning_rate = 0.001
epochs = 20
row_length = 150
init_matrix_bias = 0.5
test_size = 0.2
train_size = 1 - test_size
min_gradient_clip = 0
max_gradient_clip = 1

# Generate Model
train_acc_list, test_acc_list, train_loss_list, test_loss_list, train_rmse_list, test_rmse_list, vocab_size, output_size = generate_model(
    df, 
    activation_used, 
    learning_rate, 
    epochs, 
    init_matrix_bias, 
    row_length, 
    test_size, 
    min_gradient_clip, 
    max_gradient_clip)

# Parameters Chosen
print('\nVocab Size of RNN:', vocab_size)
print('Depth Size of RNN:', output_size)
print('Activation Function:', activation_used.capitalize())
print('Output Function:', 'Softmax')
print('Number of Epochs:', epochs)
print('Learning Rate:', learning_rate)
print('Min/Max String Length:', f'[{min_length}, {max_length}]')

# Results
print('\nTrain/Test Split = ', f'{int(train_size * 100)}:{int(test_size * 100)}')
print('Size of Dataset =', row_length)
print('Training Accuracy =', f'{round(max(train_acc_list) * 100, 2)}%')
print('Testing Accuracy =', f'{round(max(test_acc_list) * 100, 2)}%')
print('Training Loss =', round(np.mean(train_loss_list), 2))
print('Testing Loss =', round(np.mean(test_loss_list), 2))
print('Training RMSE =', round(np.sum(train_rmse_list), 2))
print('Testing RMSE =', round(np.sum(test_rmse_list), 2))
print()

# Create plots
# Training Accuracy
plot_title = 'Accuracy per Epoch for Training Data'
sub_title = f'Activation: {activation_used.capitalize()}, Epochs: {epochs}, Depth: {output_size}, Learning Rate: {learning_rate}, Instances: {row_length}, Vocab Size: {vocab_size}'
lineplot(train_acc_list, title=plot_title, sub_title=sub_title, plot_type='Accuracy', y_label='Accuracy (%)', x_label='Epochs')

# Testing Accuracy
plot_title = 'Accuracy per Epoch for Testing Data'
lineplot(test_acc_list, title=plot_title, sub_title=sub_title, plot_type='Accuracy', y_label='Accuracy (%)', x_label='Epochs')

# Training Loss
plot_title = 'Loss per Epoch for Training Data'
lineplot(train_loss_list, title=plot_title, sub_title=sub_title, plot_type='Loss', y_label='Loss', x_label='Epochs')

# Testing Loss
plot_title = 'Loss per Epoch for Testing Data'
lineplot(test_loss_list, title=plot_title, sub_title=sub_title, plot_type='Loss', y_label='Loss', x_label='Epochs')

# Training RMSE
plot_title = 'RMSE per Epoch for Training Data'
lineplot(train_rmse_list, title=plot_title, sub_title=sub_title, plot_type='RMSE', y_label='RMSE', x_label='Epochs')

# Testing RMSE
plot_title = 'RMSE per Epoch for Testing Data'
lineplot(test_rmse_list, title=plot_title, sub_title=sub_title, plot_type='RMSE', y_label='RMSE', x_label='Epochs')